In [ ]:
! pip install mesa --quiet

In [ ]:
# Librerías de simulación multiagentes
from mesa import Agent, Model  # Define los agentes y el modelo base de la simulación
from mesa.space import MultiGrid, SingleGrid  # Permite colocar los agentes en una cuadrícula (multi o individual)
from mesa.time import SimultaneousActivation  # Activa a todos los agentes simultáneamente en cada paso de la simulación
from mesa.datacollection import DataCollector  # Recolecta y organiza datos de la simulación para análisis

# Librerías matemáticas y generación de aleatoriedad
import itertools  # Proporciona herramientas para crear combinaciones y permutaciones
import random  # Permite generar números y secuencias aleatorias, útil para la variabilidad en simulaciones
import math  # Contiene funciones matemáticas básicas, como operaciones trigonométricas y logarítmicas

# Librerías de visualización y gráficos
import matplotlib.pyplot as plt  # Creación y personalización de gráficos
import matplotlib.patches as patches  # Añade formas como rectángulos o círculos a gráficos de matplotlib
from matplotlib.colors import ListedColormap  # Define mapas de color personalizados
import matplotlib.animation as animation  # Crea animaciones, útil para representar dinámicamente la simulación
plt.rcParams["animation.html"] = "jshtml"  # Configura la visualización de animaciones en formato HTML
import matplotlib  # Configuración adicional de matplotlib
matplotlib.rcParams["animation.embed_limit"] = 2**128  # Ajusta el límite de tamaño para incrustar animaciones

# Librerías de manipulación y análisis de datos
import numpy as np  # Proporciona funciones para manejo de matrices y álgebra lineal, común en análisis de datos
import pandas as pd  # Ofrece estructuras de datos como DataFrames, útiles para manipular grandes cantidades de datos
import time  # Proporciona funciones para trabajar con fechas y horas

# Librerías de comunicación en red y servidor HTTP
from http.server import BaseHTTPRequestHandler, HTTPServer  # Implementa un servidor HTTP básico para manejar solicitudes
import logging  # Registro de eventos y errores del servidor, útil para el diagnóstico
import json  # Proporciona funciones para manejar datos en formato JSON, útil para comunicación entre aplicaciones

import heapq

In [ ]:
class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.action_points = 4
        self.saved_points = 0  # Puntos ahorrados de turnos anteriores
        self.carrying_portrait = False  # Para saber si está cargando un retrato verdadero

    def calculate_action_points(self):
        """Calcula los puntos de acción disponibles al inicio del turno."""
        self.action_points = 4 + self.saved_points # Sumar puntos ahorrados
        if self.action_points > 8:  # No puede exceder 8 puntos
            self.action_points = 8
        self.saved_points = 0  # Los puntos ahorrados se usan al inicio del turno

    def move(self, direction):
        """Mueve al agente en una dirección, si tiene puntos suficientes."""
        if self.action_points >= 1:
            # Implementar lógica de movimiento aquí, según el modelo
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para moverse.")

    def open_close_door(self, door):
        """Abre o cierra una puerta."""
        if self.action_points >= 1:
            # Implementar lógica de abrir/cerrar puerta aquí
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para abrir/cerrar la puerta.")
    
    def break_wall(self, wall):
        """Rompe una pared."""
        if self.action_points >= 2:
            # Implementar lógica de romper pared aquí
            self.action_points -= 2
        else:
            print("No hay suficientes puntos de acción para romper la pared.")

    def knock_out_ghost(self, ghost):
        """Derriba un fantasma."""
        if self.action_points >= 1:
            # Implementar lógica para derribar al fantasma
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para noquear al fantasma.")

    def kill_ghost(self, ghost):
        """Mata un fantasma."""
        if self.action_points >= 2:
            # Implementar lógica para matar al fantasma
            self.action_points -= 2
        else:
            print("No hay suficientes puntos de acción para matar al fantasma.")

    def examine_portrait(self, portrait):
        """Examina un retrato."""
        if self.action_points >= 1:
            # Implementar lógica para examinar retrato
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para examinar el retrato.")

    def move_with_portrait(self, direction):
        """Se mueve cargando un retrato."""
        if self.carrying_portrait:
            if self.action_points >= 2:
                # Implementar lógica para moverse cargando el retrato
                self.action_points -= 2
            else:
                print("No hay suficientes puntos de acción para moverse cargando un retrato.")
        else:
            print("No está cargando ningún retrato.")

    def decide_to_save_points(self):
        """Decide si ahorrar puntos de acción basándose en la estrategia."""
        # Condición limitante: no puede tener más de 8 puntos
        if self.action_points >= 8:
            return False # No puede ahorrar
        
        # Prioridad 1: Si está cargando un retrato, mover hacia la salida es crítico
        if self.carrying_portrait:
            return False  # No ahorra puntos, necesita moverse hacia la salida.

        # Prioridad 2: Hay fuego cerca de víctimas o áreas importantes
        fire_nearby = self.model.check_fire_nearby(self.pos)
        if fire_nearby:
            return False  # Actuar en fuego cercano antes de ahorrar puntos.

        # Prioridad 3: Si no hay acciones urgentes, ahorrar puntos
        if self.action_points <= 2:
            # Si hay pocos puntos restantes, es más estratégico ahorrar
            return True

        # Prioridad 4: Condiciones generales para ahorrar
        return True  # Por defecto, ahorrar puntos si no hay prioridades críticas.

    def end_turn(self):
        """Termina el turno, evaluando si debe ahorrar puntos."""
        if self.decide_to_save_points():
            self.saved_points += self.action_points
            if self.saved_points > 8:  # Limitar a un máximo de 8 puntos
                self.saved_points = 8
        self.action_points = 0  # Reinicia los puntos para el siguiente turno
    
    def step(self):
        """Define el comportamiento del agente en cada paso."""
        self.calculate_action_points()

        # Priorizar acciones según el entorno
        while self.action_points > 0:
            # Aquí se puede incluir lógica de movimiento o interacción
            break
        # Finaliza el turno:
        self.end_turn()

In [ ]:
class MansionModel(Model):
    def __init__(self, width, height, walls, fake_alarms, victims, fires, doors, doors_connected, entrances):
        super().__init__()
        self.steps = 0
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(self.width, self.height, torus=False)

        # Datos de configuración
        self.walls = walls  # Paredes
        self.fake_alarms = fake_alarms  # Falsas alarmas
        self.victims = victims  # Víctimas
        self.fires = [(r - 1, c - 1) for r, c in fires]  # Coordenadas del fuego ajustadas a índices
        self.doors = doors  # Puertas
        self.doors_connected = doors_connected  # Conexiones de puertas
        self.entrances = [(r - 1, c - 1) for r, c in entrances]  # Entradas ajustadas a índices

        # Datos del modelo
        self.rescued = 0
        self.losses = 0
        self.damage = 0
        self.status = "In progress"

        # Inicializar elementos en el tablero
        self._initialize_walls()
        self._initialize_fires()
        self._initialize_points_of_interest()
        self._initialize_agents()

    def initialize_walls(self):
        for y, row in enumerate(self.walls):
            for x, cell in enumerate(row):
                if any(cell):  # Si hay alguna pared (arriba, izquierda, abajo, derecha)
                    self.grid.place_agent(LuigiAgent(f"Wall-{x}-{y}", self, "wall"), (x, y))

    def initialize_fires(self):
        for pos in self.fires:
            self.grid.place_agent(LuigiAgent(f"Fire-{pos}", self, "fire"), pos)

    def initialize_POI(self):
        for pos in self.fake_alarms:
            self.grid.place_agent(LuigiAgent(f"FakeAlarm-{pos}", self, "fake_alarm"), (pos[1] - 1, pos[0] - 1))
        for pos in self.victims:
            self.grid.place_agent(LuigiAgent(f"Victim-{pos}", self, "victim"), (pos[1] - 1, pos[0] - 1))

    def initialize_agents(self):
        for i, pos in enumerate(self.entrances):
            agent = LuigiAgent(f"Agent-{i}", self, "rescuer")
            self.grid.place_agent(agent, pos)
            self.schedule.add(agent)

    def fire(self):
        new_fires = []
        for pos in self.fires:
            neighbors = self.grid.get_neighborhood(pos, moore=False, include_center=False)
            for neighbor in neighbors:
                if neighbor not in self.fires and random.random() < 0.3:
                    new_fires.append(neighbor)
        for fire_pos in new_fires:
            self.grid.place_agent(LuigiAgent(f"Fire-{fire_pos}", self, "fire"), fire_pos)
            self.fires.append(fire_pos)

    def status(self):
        if self.losses >= 4 or self.damage >= 24:
            self.status = "Defeat"
        elif self.rescued >= len(self.victims):
            self.status = "Victory"

In [2]:
def procesar_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Procesar paredes
    WALLS = []
    index = 0
    for _ in range(6):  # 6 líneas de paredes
        row = []
        cells = lines[index].strip().split()
        for cell in cells:
            walls = [int(d) for d in cell]  # Cada celda se descompone en 4 dígitos
            row.append(walls)
        WALLS.append(row)
        index += 1

    # Procesar puntos de interés
    FAKE_ALARMS = []
    VICTIMS = []
    for _ in range(3):  # Máximo 3 líneas para puntos de interés
        r, c, marker_type = lines[index].strip().split()
        if marker_type == 'f':  # Falsa alarma
            FAKE_ALARMS.append((int(r), int(c)))
        elif marker_type == 'v':  # Víctima
            VICTIMS.append((int(r), int(c)))
        index += 1

    # Procesar marcadores de fuego
    FIRES = []
    for _ in range(10):  # 10 líneas de fuego
        r, c = map(int, lines[index].strip().split())
        FIRES.append((r, c))
        index += 1

    # Procesar marcadores de puertas
    DOORS = {}
    DOORS_CONNECTED = {}
    for _ in range(8):  # 8 líneas de puertas
        r1, c1, r2, c2 = map(int, lines[index].strip().split())
        r1, c1, r2, c2 = r1, c1, r2, c2
        DOORS[(r1, c1, r2, c2)] = (r1, c1, r2, c2)
        DOORS_CONNECTED[(r1, c1)] = (r2, c2)
        DOORS_CONNECTED[(r2, c2)] = (r1, c1)
        index += 1

    # Procesar puntos de entrada
    ENTRANCES = []
    for _ in range(4):  # 4 líneas de puntos de entrada
        r, c = map(int, lines[index].strip().split())
        ENTRANCES.append((r, c))
        index += 1

    return WALLS, FAKE_ALARMS, VICTIMS, FIRES, DOORS, DOORS_CONNECTED, ENTRANCES

# Ruta del archivo
file_path = './final.txt'

# Llamar a la función
WALLS, FAKE_ALARMS, VICTIMS, FIRES, DOORS, DOORS_CONNECTED, ENTRANCES = procesar_txt(file_path)

# Imprimir los resultados
print("Paredes:")
for row in WALLS:
    print(row)

print("\nFalsas alarmas:", FAKE_ALARMS)
print("\nVíctimas:", VICTIMS)
print("\nFuegos:", FIRES)
print("\nPuertas:", DOORS)
print("\nPuertas conectadas:", DOORS_CONNECTED)
print("\nEntradas:", ENTRANCES)

Paredes:
[[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 1, 1, 1], [1, 1, 0, 1], [1, 1, 0, 1]]
[[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 1], [0, 1, 0, 0], [0, 0, 0, 1]]
[[1, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 0], [1, 0, 1, 1], [1, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]
[[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 1, 0], [1, 0, 1, 1]]
[[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1]]
[[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]

Falsas alarmas: [(2, 4)]

Víctimas: [(6, 7), (4, 8)]

Fuegos: [(1, 7), (4, 3), (5, 2), (1, 5), (2, 2), (2, 6), (6, 1), (1, 8), (2, 1), (6, 4)]

Puertas: {(1, 6, 1, 7): (1, 6, 1, 7), (2, 6, 2, 7): (2, 6, 2, 7), (3, 6, 4, 6): (3, 6, 4, 6), (4, 6, 4, 7): (4, 6, 4, 7), (4, 5, 5, 5): (4, 5, 5, 5), (4, 6, 5, 6): (4, 